Date: 2024/11/28
Creator: Sarah You
Objective: The code in this file converts the credit history key developments gathered from S&P Cap IQ to credit rating migration matrices, specifically for the S&P 500

In [1]:
#Import required libraries
import os, glob
import pandas as pd
import re
from datetime import datetime

In [2]:
# Get the base directory by moving up two levels from the current script location
script_dir = os.getcwd()
base_dir = os.path.abspath(os.path.join(script_dir, '..', '..'))

In [3]:
#Search for Excel files
file_name = 's&p-500'
# Build the path to the specific file directory
data_dir = os.path.join(base_dir, 'credit_history_reports')
# Create a pattern to match Excel files in the directory
pattern = os.path.join(data_dir, f'{file_name}.xls')
print(pattern)
# Get a list of all Excel files matching the pattern
excel_files = glob.glob(pattern)
print(f"Found Excel files: {excel_files}")

c:\Users\wwwia\Documents\Thesis stuff\data\credit_history_reports\s&p-500.xls
Found Excel files: ['c:\\Users\\wwwia\\Documents\\Thesis stuff\\data\\credit_history_reports\\s&p-500.xls']


In [4]:
#Testing credit matrix generation on a single company. I picked AMD as it has a very long credit history.
one_file = str(excel_files[0])
whole_file = pd.ExcelFile(one_file)
test_sheet = whole_file.sheet_names[3]

In [5]:
#Extracting one test dataframe
df = pd.read_excel(one_file, sheet_name = test_sheet)
df.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Accenture plc (NYSE:ACN) > Key Developments,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Key Developments,NaN,NaN,NaN,NaN,NaN
3,Companies,Date,Event Type,Headline,Situation,Source
4,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,"Accenture plc, Standard & Poor's Upgrade Issue...",Standard & Poor's Rating Services
5,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,"Accenture plc, Standard & Poor's Upgrade Issue...",Standard & Poor's Rating Services


In [6]:
#Drop first three rows and last two coumns
df = df.drop([0,1,2]).reset_index(drop=True)
df.columns = df.iloc[0]
df.drop(df.index[0], inplace=True)
df.head(10)

,Companies,Date,Event Type,Headline,Situation,Source
1,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,"Accenture plc, Standard & Poor's Upgrade Issue...",Standard & Poor's Rating Services
2,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,"Accenture plc, Standard & Poor's Upgrade Issue...",Standard & Poor's Rating Services


In [7]:
df.drop(["Situation","Source"], axis=1, inplace=True)
df.head(10)

,Companies,Date,Event Type,Headline
1,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...
2,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...


In [8]:
#Split the Headline into Current Credit Rating and Previous Credit Rating

# Long-term ratings list (already known)
long_term_ratings = ['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'CCC', 'CC', 'C', 'D', 'NR']

# Short-term ratings list (example set; adjust as needed)
short_term_ratings = [
    'A-1', 'A-2', 'A-3', 'B', 'C', 'D'
]

# Build a pattern for long-term ratings: base + optional plus or minus
long_term_pattern = r'(?<!\w)(?:' + '|'.join(re.escape(r) for r in long_term_ratings) + r')(?:[+-])?(?!\w)'

# Build a pattern for short-term ratings directly from the list
short_term_pattern = r'(?<!\w)(?:' + '|'.join(re.escape(r) for r in short_term_ratings) + r')(?!\w)'

def extract_ratings(text):
    # Extract all long-term ratings
    lt_matches = re.findall(long_term_pattern, text)
    
    # Extract all short-term ratings
    st_matches = re.findall(short_term_pattern, text)

    # Logic to determine which rating to place in which column
    # Priority logic: 
    #  1. If we have at least one long-term rating, that becomes the New Credit Rating.
    #  2. The next available long-term rating (if any) becomes the Previous Credit Rating.
    #  3. If no second long-term rating, look at short-term ratings for the Previous Credit Rating.

    if lt_matches:
        new_rating = lt_matches[0]
        if len(lt_matches) > 1:
            prev_rating = lt_matches[1]
        else:
            # No second long-term rating found, use short-term if available
            prev_rating = st_matches[0] if st_matches else 'NR'
    else:
        # No long-term rating found, try short-term
        new_rating = st_matches[0] if st_matches else 'NR'
        prev_rating = st_matches[1] if len(st_matches) > 1 else 'NR'

    return pd.Series({'New Credit Rating': new_rating, 'Previous Credit Rating': prev_rating})

# Assuming df has the column 'Headline'
df[['New Credit Rating', 'Previous Credit Rating']] = df['Headline'].apply(extract_ratings)

In [9]:
df.head(10)

,Companies,Date,Event Type,Headline,New Credit Rating,Previous Credit Rating
1,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+
2,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+


In [10]:
#Drop columns for short term rates (so like A-1 or B-2) and drop redundant columns (same date, same rating, just another currency)

#Drop short term rates:
full_rates = ['AAA','AA+','AA','AA-','A+','A','A-',
              'BBB+','BBB','BBB-','BB+','BB','BB-','B+','B','B-',
              'CCC+','CCC','CCC-','CC+','CC','CC-','C+','C','C-',
              'D','NR']

#for i, row in df.iterrows():
#    if (row['New Credit Rating'] not in full_rates) | (row['Previous Credit Rating'] not in full_rates):
#        df.drop(i, inplace=True)
#df.shape

# Create a mask that selects rows where both 'New Credit Rating' and 'Previous Credit Rating'
# are present in long-term credit rates
mask = df['New Credit Rating'].isin(full_rates) & df['Previous Credit Rating'].isin(full_rates)
# Keep only the rows that match the condition
df = df[mask]

In [11]:
df.head(10)

,Companies,Date,Event Type,Headline,New Credit Rating,Previous Credit Rating
1,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+
2,Accenture plc (NYSE:ACN),May-10-2021 01:55 PM,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+


In [12]:
def convert_date(text):
    #Parse string into datetime context
    date_obj = datetime.strptime(text, "%b-%d-%Y %I:%M %p")
    #Format datetime object to MM/YYYY
    format_date = date_obj.strftime("%Y/%m")
    return format_date

df['Date'] = df['Date'].apply(convert_date)

df.head(10)

,Companies,Date,Event Type,Headline,New Credit Rating,Previous Credit Rating
1,Accenture plc (NYSE:ACN),2021/05,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+
2,Accenture plc (NYSE:ACN),2021/05,Credit Rating - S&P - Upgrade,Issuer Credit Rating: AA-; Stable from A+; Sta...,AA-,A+


In [13]:
# Drop duplicates based on Date, New Credit Rating and Previous Credit Rating
df = df.drop_duplicates(subset=['Date','New Credit Rating','Previous Credit Rating'], keep='first')
#Drop Event Type and Headline
df = df.drop(['Event Type','Headline'],axis=1).reset_index(drop=True)
df.head(10)

,Companies,Date,New Credit Rating,Previous Credit Rating
0,Accenture plc (NYSE:ACN),2021/05,AA-,A+


In [14]:
#Filter for dates after 01/01/2000
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m')
df = df.sort_values(by='Date').reset_index(drop=True)

df.head(10)

,Companies,Date,New Credit Rating,Previous Credit Rating
0,Accenture plc (NYSE:ACN),2021-05-01,AA-,A+


In [15]:
#The hard part: adding all the other months
#Add dates

#Add initial rating if first date in column is after 2000-01-01:
initial_date = pd.to_datetime('2000-01-01')
if df['Date'].iloc[0] > initial_date:
    initial_rating = {
        'Companies': [df['Companies'].iloc[0]],
        'Date': [initial_date],
        'New Credit Rating': ['NR'],
        'Previous Credit Rating': ['NR']
    }
    rating_history = pd.concat([pd.DataFrame(initial_rating), df], ignore_index=True)
    rating_history.sort_values(by='Date').reset_index(drop=True)
else:
    rating_history = df

# Create the full date range DataFrame
date_range = pd.date_range(start='2000-01-01', end='2023-12-01', freq='MS')
companies = df['Companies'].unique()
# Create a DataFrame with all combinations of companies and dates
full_df = pd.MultiIndex.from_product([companies, date_range], names=['Company', 'Date']).to_frame(index=False)
full_df = full_df.sort_values(by='Date')
rating_history = rating_history.rename(columns={'Date': 'Effective Date', 'Companies':'Company'})
# Perform the 'asof' merge
merged_df = pd.merge_asof(
    full_df,
    rating_history,
    left_on='Date',
    right_on='Effective Date',
    by='Company',
    direction='backward'
)

# Adjust 'Previous Credit Rating' after the last known change
# Get the last 'Effective Date' for each company
last_effective_date = rating_history.groupby('Company')['Effective Date'].max().reset_index()
last_effective_date.columns = ['Company', 'Last Effective Date']

# Merge and adjust
merged_df = pd.merge(merged_df, last_effective_date, on='Company', how='left')
merged_df.loc[merged_df['Date'] > merged_df['Last Effective Date'], 'Previous Credit Rating'] = merged_df['New Credit Rating']

# Step 6: Adjust 'Previous Credit Rating' where 'Credit Rating' hasn't changed
def adjust_previous_rating(group):
    group = group.sort_values('Date').reset_index(drop=True)
    group['Credit Rating Shift'] = group['New Credit Rating'].shift(1)
    # Initialize the first 'Credit Rating Shift' as the same as 'Credit Rating'
    group.loc[0, 'Credit Rating Shift'] = group.loc[0, 'New Credit Rating']
    group['Rating Changed'] = group['New Credit Rating'] != group['Credit Rating Shift']
    group['Previous Credit Rating'] = group.apply(
        lambda row: row['Previous Credit Rating'] if row['Rating Changed'] else row['New Credit Rating'],
        axis=1
    )
    return group

merged_df = merged_df.groupby('Company').apply(adjust_previous_rating).reset_index(drop=True)

# Step 8: Prepare the final DataFrame
final_df = merged_df[['Company', 'Date', 'New Credit Rating', 'Previous Credit Rating']]

final_df.sample(10)

C:\Users\wwwia\AppData\Local\Temp\ipykernel_18680\62086580.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('Company').apply(adjust_previous_rating).reset_index(drop=True)


,Company,Date,New Credit Rating,Previous Credit Rating
182,Accenture plc (NYSE:ACN),2015-03-01,NR,NR
27,Accenture plc (NYSE:ACN),2002-04-01,NR,NR
258,Accenture plc (NYSE:ACN),2021-07-01,AA-,AA-
286,Accenture plc (NYSE:ACN),2023-11-01,AA-,AA-
95,Accenture plc (NYSE:ACN),2007-12-01,NR,NR
150,Accenture plc (NYSE:ACN),2012-07-01,NR,NR
105,Accenture plc (NYSE:ACN),2008-10-01,NR,NR
222,Accenture plc (NYSE:ACN),2018-07-01,NR,NR
276,Accenture plc (NYSE:ACN),2023-01-01,AA-,AA-
11,Accenture plc (NYSE:ACN),2000-12-01,NR,NR


In [16]:
final_df.head(10)

,Company,Date,New Credit Rating,Previous Credit Rating
0,Accenture plc (NYSE:ACN),2000-01-01,NR,NR
1,Accenture plc (NYSE:ACN),2000-02-01,NR,NR
2,Accenture plc (NYSE:ACN),2000-03-01,NR,NR
3,Accenture plc (NYSE:ACN),2000-04-01,NR,NR
4,Accenture plc (NYSE:ACN),2000-05-01,NR,NR
5,Accenture plc (NYSE:ACN),2000-06-01,NR,NR
6,Accenture plc (NYSE:ACN),2000-07-01,NR,NR
7,Accenture plc (NYSE:ACN),2000-08-01,NR,NR
8,Accenture plc (NYSE:ACN),2000-09-01,NR,NR
9,Accenture plc (NYSE:ACN),2000-10-01,NR,NR


In [17]:
#Collapse the information into onw row
df_wide = final_df.pivot(index='Company', columns='Date', values='New Credit Rating')

df_wide

Date,2000-01-01,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01,2000-07-01,2000-08-01,2000-09-01,2000-10-01,...,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
Company,,,,,,,,,,,,,,,,,,,,,
Accenture plc (NYSE:ACN),NR,NR,NR,NR,NR,NR,NR,NR,NR,NR,...,AA-,AA-,AA-,AA-,AA-,AA-,AA-,AA-,AA-,AA-
